# Import

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import time
import pyvisa # control gpib, tcp/ip etc...
import nidaqmx # Control ni-usb-6002
import paramiko
from scp import SCPClient   
import os
from matplotlib.animation import FuncAnimation
from itertools import count
import cv2
import natsort
import scipy
import lmfit

######### Module for c++
# import clr
# from System import Decimal

# for APT control of thorlab stages
from pylablib.devices import Thorlabs

# After rm_listsource does not show USB's....This is a longer way to take control over PM160
from ctypes import cdll,c_long, c_ulong, c_uint32,byref,create_string_buffer,c_bool,c_char_p,c_int,c_int16,c_double, sizeof, c_voidp
from TLPM import TLPM # TLPM.py is required

# Instruments

In [6]:
rm = pyvisa.ResourceManager()

In [7]:
rm.list_resources()

('USB0::0x1313::0x807B::16041103::INSTR',
 'TCPIP0::192.168.0.2::inst0::INSTR',
 'ASRL1::INSTR',
 'ASRL4::INSTR',
 'ASRL5::INSTR',
 'ASRL6::INSTR',
 'ASRL7::INSTR',
 'ASRL8::INSTR',
 'GPIB0::6::INSTR')

In [8]:
# load agilent
smf100a=rm.open_resource('TCPIP::192.168.0.2::hislip0::INSTR');time.sleep(0.5) 

# load ghost
ghost=rm.open_resource('TCPIP::localhost::4000::SOCKET');time.sleep(1)
ghost.read_termination='\r\n';time.sleep(1)

# load Kepco
kepco=rm.open_resource('GPIB0::6::INSTR');time.sleep(0.5)

# load VNA
pna5230a=rm.open_resource('GPIB0::5::INSTR');time.sleep(0.5)

# User defined functions

In [12]:
##############################   kepco  ########################################         
def init_kepco_local():
    kepco.write('FUNC:MODE CURR')
    kepco.write('CURR 0.00000')
    kepco.write('OUTP ON')
    return

def change_curr_local(curr):
    kepco.write('CURR %.4f'%curr)
    return

##############################   PSP  ######################################## 
def change_volt_PSP(volt):
    PSP603.write('SV %.2f'%volt);
    return volt

def PSP_seq_volt(volt):
    tlist=np.linspace(0,5,100)
    vlist=volt+volt*np.sin(tlist*2*np.pi/0.2)*np.exp(-tlist**2/3)
    tstep=(tlist[1]-tlist[0])
    print('PSP voltage stablization is running...')
    for i,v in enumerate(vlist):
        change_volt_PSP(v);time.sleep(tstep)

##############################   lockin  ######################################## 
def getAvg(sens):
    with nidaqmx.Task() as task:
        task.ai_channels.add_ai_voltage_chan("Dev1/ai2") # dev2 is multi-purpose ni-daq card: AI2=x & AI3=y
#         task.timing.cfg_samp_clk_timing(1000)
        data=task.read(number_of_samples_per_channel=1024)
        return np.mean(data[800:])*sens/10.0 # sens in mV
def getAvgY(sens):
    with nidaqmx.Task() as task:
        task.ai_channels.add_ai_voltage_chan("Dev1/ai3") # dev2 is multi-purpose ni-daq card: AI2=x & AI3=y
#         task.timing.cfg_samp_clk_timing(1000)
        data=task.read(number_of_samples_per_channel=1024)
        return np.mean(data[800:])*sens/10.0 # sens in mV
    
def getAvg_GPIB(i):
    return float(sr830a.query('OUTP? %i'%i).split('\n')[0])

def getAvg_GPIB_TRAC():
    sr830a.write('REST')
    sr830a.write('STRD')
    time.sleep(1)
    return np.mean([float(x) for x in sr830a.query('TRCA? 1,0,20').split(',')[:-1]])    
# second lockin for Pt voltage
def getlock2x(sens):
    with nidaqmx.Task() as task:
        task.ai_channels.add_ai_voltage_chan("Dev1/ai4") # dev2 is multi-purpose ni-daq card: AI2=x & AI3=y
#         task.timing.cfg_samp_clk_timing(1000)
        data=task.read(number_of_samples_per_channel=1024)
        return np.mean(data[800:])*sens/10.0 # sens in mV
    
def getlock2y(sens):
    with nidaqmx.Task() as task:
        task.ai_channels.add_ai_voltage_chan("Dev1/ai5") # dev2 is multi-purpose ni-daq card: AI2=x & AI3=y
#         task.timing.cfg_samp_clk_timing(1000)
        data=task.read(number_of_samples_per_channel=1024)
        return np.mean(data[800:])*sens/10.0 # sens in mV
    
def adjust_sens():
    senslist=np.array([2e-6,5e-6,10e-6,20e-6,50e-6,100e-6,200e-6,500e-6,1e-3,2e-3,5e-3,10e-3,20e-3,50e-3,100e-3,200e-3,500e-3,\
                  1,2,5,10,20,50,100,200,500,1000])
    
    init_sens=14 # 16 = 0.1 mV
    sr830a.write('SENS %i'%(init_sens))
    print(f'Sens is adjusted to {senslist[init_sens]} mV...')
    time.sleep(0.1)
    i=0
    while float(sr830a.query('LIAS? 2').split('\n')[0])==1:
        i=i+1
        sr830a.write('SENS %i'%(init_sens+i))
        print(f'Sens is adjusted to {senslist[init_sens+i]} mV...')
        time.sleep(0.1)
#         float(sr830a.query('LIAS? 2').split('\n')[0])#clear bit 2
    sr830a.write('SENS %i'%(init_sens+i+1))
    print(f'Sens is adjusted to {senslist[init_sens+i+1]} mV...')
    time.sleep(0.1)
    return senslist[init_sens+i+1]    

##############################   microwave  ######################################## 
def change_power(power):
    smf100a.write('SOUR:POW %.10f'%power)
    return

def turnMW_on_off(control):
    smf100a.write("OUTP:All %s"%control)
    return

def change_freq(freq):
    smf100a.write('SOUR:FREQ %.10f'%freq)
    return freq

def change_mod(control):
    smf100a.write('SOUR:Mod %s'%control)
    return

def init_LFO():               
    smf100a.write('SOUR:LFO1:VOLT %.10f'%2.5) # 2.5 is good for simulating TTL for lockin ref
    smf100a.write('SOUR:LFO1 ON')
    smf100a.write('SOUR:LFO1:OFFS 0')
def set_LFO_pulse(wid,peri):   
    
    smf100a.write('SOUR:LFO1:SHAP:PULS:WIDT %.10f'%wid)
    smf100a.write('SOUR:LFO1:SHAP:PULS:PER %.10f'%peri)
    
##############################   file i/o  ######################################## 
def save_header(pfile, outputfile, header, flag='w') :
    with open(pfile) as fp:
        content = fp.readlines()
        fp.close()
    with open(outputfile, flag) as fp:
        for line in content:
            print('#', line[:-1], file=fp)
        if flag == 'w': print('#', header, file=fp)
        fp.close()
    return
    
def stopper(pfile):
    with open(pfile) as fp:
        content = fp.readlines()
        fp.close()
    return content[0].split('\n')[0]

def read_linenum(pfile):
    with open(pfile) as fp:
        content = fp.readlines()
        fp.close()
        i=0
        while len(content[-i-1].split('::::'))==1:
            i=i+1
    return int(content[-i-2].split(':')[0])

##############################   ghost  ######################################## 
def check_ghost_acquiring(sweepcount,BLScount):
    global error_occured
    error_occured=0
    status='Template:  on'
    while status.split(':')[1].split('on')[0]!=' IDLE ':
        ghost.clear();time.sleep(0.1)
        ghost.write("status");time.sleep(0.1)
        for i in range(2):
            status=ghost.read('\n');time.sleep(0.1)
            print('%i: %s'%(i,status))

        if len(status.split(':'))==1:
            print('Status returned null...') # Emergency reset
            print('ghost message is %s'%status)
            ghost.clear();time.sleep(1);print('clear requested')
            ghost.write('stop');time.sleep(1);print('stop requested')
            ghost.write('restore');time.sleep(1);print('resotred')
            ghost.write('override');time.sleep(3);print('overrode')
            ghost.clear();time.sleep(1);print('2nd clear requested')
            ghost.write('delete');time.sleep(0.1);print('delete requested')
#             for i in range(1):
#                 status=ghost.read('\n');time.sleep(0.1)
#                 print('%i: %s'%(i,status))
            ghost.write('stop');time.sleep(1);print('stop requested')
            ghost.write('delete');time.sleep(0.1);print('delete requested')
            ghost.clear();time.sleep(1);print('clear requested')
            ghost.write('delete');time.sleep(1);print('delete requested')
            ghost.write('start %i'%BLScount);time.sleep(1)
            print('starting scan number %i again...'%sweepcount)
            status='Template:  on'
            error_occured=error_occured+1
            
        if len(status.split('<infinite>'))==2:
            print('Status entered infinity loop...') 
            ghost.clear();time.sleep(1);print('clear requested')
            ghost.write('stop');time.sleep(1);print('stop requested')
            ghost.write('restore');time.sleep(1);print('resotred')
            ghost.write('override');time.sleep(3);print('overrode')
            ghost.clear();time.sleep(1);print('2nd clear requested')
            ghost.write('delete');time.sleep(1);print('delete requested')
            ghost.write('start %i'%BLScount);time.sleep(0.1)
            print('starting scan number %i again...'%sweepcount)
            status='Template:  on'
            error_occured=error_occured+1
        else:
            pass
    return error_occured

def get_zerodata(month,fname):
    motherpath='D:/Jupyter_notebooks/Kyongmo/'
    path=motherpath+month+'/'+'BLS_'+fname+'/'
    files=natsort.natsorted(os.listdir(path))
    data=np.loadtxt(path+files[0],skiprows=12)
    return data[len(data)//2]
def get_latestdata(month,fname):
    motherpath='D:/Jupyter_notebooks/Kyongmo/'
    path=motherpath+month+'/'+'BLS_'+fname+'/'
    files=natsort.natsorted(os.listdir(path))
    data=np.loadtxt(path+files[-1],skiprows=12)
    return data[len(data)//2]

def get_BLS_Singleshot_counts(fname,month):
#     ghost.write('wdir D:/Jupyter_notebooks/Kyongmo/2304/BLS_%s'%fname);time.sleep(0.1)
    ghost.write('delete');time.sleep(0.1)
    ghost.write('start %i'%1);#time.sleep(90/100*BLScount)
    error_occured=check_ghost_acquiring(1,1);time.sleep(0.1)
    ghost.write('stop');time.sleep(0.1)
    ghost.write('save BLS_singleshot.dat');time.sleep(0.1)
    
    motherpath='D:/Jupyter_notebooks/Kyongmo/'
    path=motherpath+month+'/'+'BLS_'+fname+'/'
    data=np.loadtxt(path+'BLS_singleshot.dat',skiprows=12)
    return np.sum(data[0:len(data)//4])

def BLS_maximizer(test_freq,test_power):
    nscan=5
    nhalf=nscan//2
    stagex.move_by(int(-nhalf*2180));time.sleep(1) #2180 is 1 um
    stagey.move_by(int(-nhalf*2180));time.sleep(1)
    xc,yc,zi=np.zeros((3,nscan,nscan))
    x0=stagex.get_position()
    y0=stagey.get_position()
    
    change_freq(test_freq)
    change_power(test_power)
    for i in range(nscan):
        if i==0:
            pass
        else:
            stagey.move_to(int(y0));
            stagex.move_by(int(2180));
        for j in range(nscan):
            if j==0:
                pass
            else:
                stagey.move_by(int(2180))
            tdata=get_BLSinten(fname,'2304')
            xc[i,j]=stagex.get_position()
            yc[i,j]=stagey.get_position()
            zi[i,j]=get_BLS_Singleshot_counts(fname,'2304')
            print('x=%.0f,y=%.0f,Total count=%.0f'%(xc[i,j],yc[i,j],zi[i,j]))
    max_index = np.argmax(zi)
    max_index_tuple = np.unravel_index(max_index, zi.shape)
    print('drift correction finished')
    maxpos=xc[max_index_tuple],yc[max_index_tuple]
    print('Max total: %.0f at x=%.0f,y=%.0f'%(zi[max_index_tuple],maxpos[0],maxpos[1]))
    stagex.move_to(int(maxpos[0]))
    stagey.move_to(int(maxpos[1]))
    print('moved to the max BLS intensity position')
    print('return to the original power: %.3f dBm'%power)
    change_power(power)
    os.remove('BLS_singleshot.dat')
    os.remove('test123.dat')
##############################   xy stage  ########################################            
def init_stage():
    ks.TLI_BuildDeviceList();time.sleep(0.5) #initialize
    ks.TLI_GetDeviceListSize();time.sleep(0.5)
    
    serialnox = c_char_p(bytes("26001463", "utf-8"))
    serialnoy = c_char_p(bytes("26001408", "utf-8"))
    
    ks.SCC_Open(serialnox);time.sleep(0.1)
    ks.SCC_Open(serialnoy);time.sleep(0.1)
    return serialnox,serialnoy

cfactor=4.577e-07
def get_posx():
    xstep=ks.SCC_GetPosition(serialnox)    
    xmm=cfactor*(xstep+3020209)-1.3825
    return xmm

def move_posx(xmm):
    xstep=(xmm+1.3825)/cfactor-3020209
    ks.SCC_MoveToPosition(serialnox, int(xstep))

def get_posy():
    ystep=ks.SCC_GetPosition(serialnoy)    
    ymm=cfactor*(ystep-1447199)+0.6625
    return ymm

def move_posy(ymm):
    ystep=(ymm-0.6625)/cfactor+1447199
    ks.SCC_MoveToPosition(serialnoy, int(ystep))
    
##############################   camera  ######################################## 
def takepic(picture_name):
    cap = cv2.VideoCapture(0)
    ret, frame = cap.read()
    out = cv2.imwrite(picture_name, frame)
    time.sleep(0.5)
    cap.release()
    cv2.destroyAllWindows()
    
##############################   laser  ######################################## 
def get_powerNOW():
    thorlabsPM16.write('READ?');time.sleep(0.1)
    powerNOW=1e3*float(thorlabsPM16.read().split()[0])
    return powerNOW

def get_powerTLPM():
    l_power =  c_double()
    tlPM.measPower(byref(l_power))
    return float(l_power.value)*1e3

def setTorus(percent):
    if 29<percent<96:
        Torus532.write('CURRENT=%.2f'%percent);time.sleep(0.2)
    else:
        print('Request power is out of range!')
        
def AdjustTorus(power_requested,initial_percent,step):
    global laser_percent
    i=0
    setTorus(initial_percent)
    if get_powerNOW()<power_requested:
        while get_powerNOW()<power_requested:
            setTorus(initial_percent+i*step);time.sleep(0.1)
            i=i+1
            print(f'Laser power is {get_powerNOW()} @ {initial_percent+i*step} percents');
            laser_percent=initial_percent+i*step
            
    else:
        while get_powerNOW()>power_requested:
            setTorus(initial_percent-i*step);time.sleep(0.1)
            i=i+1
            print(f'Laser power is {get_powerNOW()} @ {initial_percent-i*step} percents');
            laser_percent=initial_percent-i*step
    return laser_percent

##############################   z stage  ######################################## 

def move2field(desired_field,step,calfactor):
    #calfactor=324.94/251.63
    current_field=float(f41.query('FETCh:DC?'))*1e3*calfactor # in mT
    if desired_field<current_field:
        while desired_field<current_field:
            stagez.move_by(int(step))
            time.sleep(0.1)
            while stagez.is_moving()==1:
                print('waiting for stop...')
                time.sleep(1)
                print(f'Z-stage is now at {stagez.get_position()}.')
                time.sleep(0.1)
                current_field=float(f41.query('FETCh:DC?'))*1e3*calfactor
                print(f'current field is {current_field} mT')
    else:
        while desired_field>current_field:
            stagez.move_by(int(-step))
            time.sleep(0.1)
            print()
            while stagez.is_moving()==1:
                print('waiting for stop...')
                time.sleep(1)
                print(f'Z-stage is now at {stagez.get_position()}.')
                time.sleep(0.1)
                current_field=float(f41.query('FETCh:DC?'))*1e3*calfactor
                print(f'current field is {current_field} mT')

def move2field_senis(desired_field,step,calfactor):
    #calfactor=324.94/251.63
    current_field=float(measureSenis())*1e0*calfactor # in mT
    if desired_field<current_field:
        while desired_field<current_field:
            stagez.move_by(int(step))
            time.sleep(0.1)
            while stagez.is_moving()==1:
                print('waiting for stop...')
                time.sleep(1)
                print(f'Z-stage is now at {stagez.get_position()}.')
                time.sleep(0.1)
                current_field=float(measureSenis())*1e0*calfactor
                print(f'current field is {current_field} mT')
    else:
        while desired_field>current_field:
            stagez.move_by(int(-step))
            time.sleep(0.1)
            print()
            while stagez.is_moving()==1:
                print('waiting for stop...')
                time.sleep(1)
                print(f'Z-stage is now at {stagez.get_position()}.')
                time.sleep(0.1)
                current_field=float(measureSenis())*1e0*calfactor
                print(f'current field is {current_field} mT')                
########################################### Senis ##################
def measureSenis():
    scp.SCPClient(conRPi.get_transport()).put('MeasureKM.py','/home/pi/software/threeDhall_sw');time.sleep(0.3)
    stdin,stdout,stderr=conRPi.exec_command('python3 /home/pi/software/threeDhall_sw/MeasureKM.py');time.sleep(1)
    scp.SCPClient(conRPi.get_transport()).get('/home/pi/software/threeDhall_sw/bzfield.dat')
    time.sleep(0.3)
    bx,by,bz,temphall=np.loadtxt('bzfield.dat')
#     print(bx,by,bz,temphall)
    return bz


############ Lockin control


header="0:Power\t 0:RF \t 2:Curr(A)\t 3:Freq \t 4:sX (V) \n"    

# Stage initilization

In [13]:
Thorlabs.list_kinesis_devices()

[('83857283', 'APT DC Motor Controller'), ('26001408', 'Stepper Controller')]

In [14]:
################# APT control of stages
# Thorlabs.list_kinesis_devices()

stagex = Thorlabs.KinesisMotor("83857283") # xstage: 100 step 4.6 um
stagey = Thorlabs.KinesisMotor("26001408") # ystage: 10000 step 4.6 um

############### Kinesis control for xy stages
# serialnox,serialnoy=init_stage()

# # get current x position
# current_motor_posx = ks.SCC_GetPosition(serialnox)
# print("Current position x:", current_motor_posx)

# # get current y position
# current_motor_posy = ks.SCC_GetPosition(serialnoy)
# print("Current position y:", current_motor_posy)

print(r'10000 step is about %.4f um'%(4.6))

10000 step is about 4.6000 um


# Single-shot commands

## Stage operations

In [21]:
stagez.setup_drive(velocity=1200,acceleration=1200)

TPZMotorDriveParams(max_voltage=112, velocity=1200, acceleration=1200)

In [17]:
stagex.move_by(int(-100))

## Microwave

In [201]:
turnMW_on_off('OFF')

In [25]:
change_mod('ON')

In [26]:
change_power(30)

In [25]:
change_freq(21e9)

21000000000.0

## Local control ghost

In [203]:
ghost.write('restore');time.sleep(1) 

## Local control kepco

In [19]:
init_kepco_local()  

In [67]:
change_curr_local(0)

In [19]:
float((kepco.query('meas:curr?')).split('\n')[0])

-0.0115

# <font color='blue'>Measure BLS</font>

In [ ]:
print('First_line_identifier::::should be kept here')
print('Sample :')
######################## Set parameters
fname='230817a3_Fe2O3_BLS_Freqsweep_30dBm_15GHz30GHz_15A'
curr_list=np.linspace(4,4,1)
freq_list=np.linspace(10e9,25e9,31)
power_list=np.array([30])
BLScount=300
# power_list=np.linspace(0.01,1,10) # in mW
# power_list=10*np.log10(power_list)
# posz_list=np.concatenate((np.linspace(1000,1000,7),np.linspace(100,100,6),np.linspace(25,25,124),\
#                           np.linspace(100,100,7),np.linspace(1000,1000,6)))
hz_list=np.linspace(0,0,1) #
posy_list=np.linspace(0,0,1) # cal: 1000 step = 0.5 um
posx_list=np.linspace(0,0,1) # better accuracy: 2140 = 1 um
laserpower_list=np.linspace(10,10,1)
modfreq_list=np.linspace(3177,3177,1)
ptcurr_list=np.linspace(0,0,1)
volt_list=np.linspace(0,0,1)
# pulse_const=0.1e-3*3177*4

calfactor=1
try:
    laser_percent
except NameError:
    print('laser_pecent is not defined and it is now 90!')
    laser_percent=84.3
percent_step=0.05 # default = 0.02
# laser_percent=84.3
sens=10 # in mV
sens2=20e-3 # in mV

###################### Set Directory
os.mkdir('D:/Jupyter_notebooks/Kyongmo/2308/BLS_%s'%fname)
ghost.write('override');time.sleep(3)
ghost.write('wdir D:/Jupyter_notebooks/Kyongmo/2308/BLS_%s'%fname)

###################### Save picture at starting position 
os.mkdir('D:/Jupyter_notebooks/Kyongmo/2308/Pictures/%s'%fname)
# takepic('D:/Jupyter_notebooks/Kyongmo/2308/Pictures/%s/begin.png'%fname)

###################### Register time
time_start=time.localtime()
print('start at : %d.%d.%d,%d:%d'%(time_start[0],time_start[1],time_start[2],time_start[3],time_start[4]))
time_start=time.time()

###################### Set swiches
sweepcount=1
dt=[]
error_occured=[]

#####################save codes
get_ipython().magic(u'history -n -f history.py')
num=read_linenum('history.py')
os.remove('history.py')
get_ipython().magic('%save -f temp.py -n {}'.format(str(num)))
save_header('temp.py',fname+'.dat', header)

####################### Loop
try:
    for f0,modfreq in enumerate(modfreq_list):
        print('Mod freq is changing...')
#         set_LFO_pulse(pulse_const/modfreq,1/modfreq)
#         sr830a.write('FREQ %.4f'%modfreq)
        time.sleep(1)

        for k1,laserpower in enumerate(laserpower_list):
            print('Laser power adjustment is starting...')
#             laser_percent_new=AdjustTorus(laserpower,laser_percent,percent_step)
#             time.sleep(1)
            
            for s2,ptcurr in enumerate(ptcurr_list):
#                 key2962A.write('CURR %3f'%ptcurr)
                time.sleep(0.01)
                
                for k2,power in enumerate(power_list):
                    print('MW Power is changing...')
                    change_power(power)
                    time.sleep(1)

                    for j,curr in enumerate(curr_list):
                        print('Kepco current is changing...')
                        change_curr_local(curr)
                        time.sleep(0.5) # stablization of field

                        for n,volt in enumerate(volt_list):
                            print('PSP voltage is changing...')
    #                         if n==0:
    #                             PSP_seq_volt(volt_list[-1]) # remove hystresis
    #                             PSP_seq_volt(volt_list[0]) # remove hystresis
    #                         PSP_seq_volt(volt)
#                             change_volt_PSP(volt)
#                             time.sleep(1) # stablization of field

                            for m1,hz in enumerate(hz_list):
#                                 move2field(**dict(desired_field=hz,step=1000,calfactor=1))
                                time.sleep(0.1)

                                for m2,posx in enumerate(posx_list):
#                                     if m2==0:
#                                         pass
#                                     else:
#                                         stagey.move_to(int(init_ypos))
#                                         print(f'x-stage is now at {stagex.get_position()}.')
#                                         stagex.move_by(int(posx));time.sleep(1)
#                                         time.sleep(1)
#                                         while stagex.is_moving()==1:
#                                             time.sleep(0.5)
#                                             print('waiting for x-stage to stop...')
#                                         time.sleep(1)
                                    
                                    for m3,posy in enumerate(posy_list):
#                                         if m3==0:
#                                             pass
#                                         else:
#                                             stagey.move_by(int(posy));time.sleep(1)
#                                             print(f'y-stage is now at {stagey.get_position()}.')
#                                             time.sleep(1)
#                                             while stagey.is_moving()==1:
#                                                 time.sleep(0.5)
#                                                 print('waiting for y-stage to stop...')
#                                             time.sleep(1)
#                                         BLS_maximizer(6.065e9,18) # scan over 4 um x 4 um square to find max
#                                         time.sleep(1)
#                                         takepic('D:/Jupyter_notebooks/Kyongmo/2308/Pictures/%s/x=%i_y=%i.png'%(fname,m2,m3))
                                        
                                        for i,freq in enumerate(freq_list):
                                            time1=time.time()
                                            print('MW Freq is changing...')
                                            change_freq(freq)
                                            time.sleep(0.07)
        #                                     thorlabsPM16.write('READ?'); # read laser power
#                                             lpowernow=get_powerNOW()
                                            ghost.write('delete');time.sleep(0.1)
                                            print('starting scan number %i...'%sweepcount)
                                            ghost.write('start %i'%BLScount);#time.sleep(90/100*BLScount)
                                            error_occured=check_ghost_acquiring(sweepcount,BLScount);time.sleep(0.1)
                                            ghost.write('stop');time.sleep(0.1)
                                            ghost.write('save %i.dat'%sweepcount);time.sleep(0.1)

                                            print('%i.dat created'%sweepcount)
                                            print('scan number %i finished.'%sweepcount)
                                            Ncol=19
                                            data_raw=np.zeros(Ncol)
                                            data_raw[0] = float(power)
                                            data_raw[1] = 0#stagex.get_position()
                                            data_raw[2] = 0#stagey.get_position()
                                            data_raw[3] = hz                   
                                            data_raw[4] = float(curr)
                                            data_raw[5] = float(freq)
                                            data_raw[6] = float((kepco.query('meas:curr?')).split('\n')[0]) #actual current
                                            data_raw[7] = 0#float(getAvg(sens))#float(getAvg()) #lockin X voltage
                                            data_raw[8] = 0#lpowernow
                                            data_raw[9] = float(volt) # PSP voltage output
                                            data_raw[10] = 0#float(f41.query('FETCh:DC?'))*1e3*1 # Field at the center in mT : 0.9172=calfactor
                                            data_raw[11] = 0#float(laserpower) # laser power
                                            data_raw[12] = float(modfreq) # modulation 
                                            data_raw[13] = float(sens)
                                            data_raw[14] = float(time1) 
                                            data_raw[15] = float(error_occured)
                                            data_raw[16] = 0#float(getlock2x(sens2))
                                            data_raw[17] = 0#float(getlock2y(sens2))
                                            data_raw[18] = float(ptcurr)
                                            status=stopper('status.py')
                                            if status=='STOP':
                                                print('Measurement has been stopped')
                                                time.sleep(1)
                                                raise StopIteration

                                            data_arr=np.array([data_raw[0],data_raw[1],data_raw[2],\
                                                               data_raw[3],data_raw[4],data_raw[5],\
                                                               data_raw[6],data_raw[7],data_raw[8],\
                                                               data_raw[9],data_raw[10],data_raw[11],\
                                                               data_raw[12],data_raw[13],data_raw[14],\
                                                               data_raw[15],data_raw[16],data_raw[17],\
                                                              data_raw[18]])
                                            data = np.reshape(data_arr,(1,Ncol))
                                            if sweepcount==1:
                                                fp=open(fname+'.dat','a')
                                            else:
                                                pass
                                            np.savetxt(fp,data, delimiter='\t', newline=os.linesep);time.sleep(0.1)
                                            fp.flush() # flush all buffer data to a file
                                            TotN=len(curr_list)*len(power_list)*len(freq_list)*len(posx_list)*len(posy_list)*len(hz_list)\
                                            *len(volt_list)*len(laserpower_list)*len(modfreq_list)*len(ptcurr_list)
                                            if sweepcount==TotN:
                                                fp.close()  
                                            print('MW Power: {} dBm, Pos x: {}, Pos y: {}, Pos z: {}, Kepco : {} A, MW freq : {} Hz, Laser Power: {} mW, Field : {}, Mod Freq : {}'.format(\
                                                    data_raw[0],data_raw[1],data_raw[2],data_raw[3], data_raw[6], data_raw[5],data_raw[8],data_raw[10],data_raw[12]))
                                            time2=time.time()
                                            dt.append((time2-time1)/60)
                                            dtmean=np.mean(dt)
                                            tleft=dtmean*(TotN-sweepcount)
                                            print('Time left : %.4f minutes'%(tleft))
                                            print('Completed scan %d out of %d'%(sweepcount,TotN))
                                            sweepcount=sweepcount+1
                                            
                                            ########## safety checks ##########
                                            # check laser power
                                            #if (data_raw[8]>laserpower*1.005)|(data_raw[8]<laserpower*0.995):
                                             #   print('Laser power adjustment is starting...')
                                              #  laser_percent=AdjustTorus(laserpower,laser_percent,percent_step)
                                            
                                            # check scan stability
#                                             zerodata=get_zerodata('2308',fname)
#                                             latestdata=get_latestdata('2308',fname)
#                                             if (latestdata>zerodata*1.5)|(latestdata<zerodata*0.5):
#                                                 print('Scan is unstable!')
#                                                 laser_percent=AdjustTorus(0.07,laser_percent,1)
#                                                 time.sleep(1)
#                                                 raise StopIteration
#                                             else:
#                                                 print('Scan works fine!')
except StopIteration:
    fp.close()
    
###################### Save picture at ending position 
# takepic('D:/Jupyter_notebooks/Kyongmo/2308/Pictures/%s/end.png'%fname)
##return to original posiiton
# stagex.move_to(int(init_xpos))
# stagey.move_to(int(init_ypos))
print('Total time elapsed %.3f mins'%((time2-time_start)/60))
ghost.write('restore');time.sleep(1)

In [40]:
ghost.write('restore');time.sleep(1)

# <font color='blue'>Measure micro-spatial scan BLS</font>

In [ ]:
print('First_line_identifier::::should be kept here')
print('Sample :')
######################## Set parameters
fname='230817c4_Fe2O3_BLS_xyscan_16A'
curr_list=np.linspace(16,16,1)
freq_list=np.linspace(10e9,10e9,1)
power_list=np.array([0])
BLScount=1000
# power_list=np.linspace(0.01,1,10) # in mW
# power_list=10*np.log10(power_list)
# posz_list=np.concatenate((np.linspace(1000,1000,7),np.linspace(100,100,6),np.linspace(25,25,124),\
#                           np.linspace(100,100,7),np.linspace(1000,1000,6)))
hz_list=np.linspace(0,0,1) #
posy_list=np.linspace(10000,10000,3) # 10000 step = 4.6 um
posx_list=np.linspace(100,100,3) # cal: 100 step = 4.6 um
laserpower_list=np.linspace(10,10,1)
modfreq_list=np.linspace(3177,3177,1)
ptcurr_list=np.linspace(0,0,1)
volt_list=np.linspace(0,0,1)
# pulse_const=0.1e-3*3177*4

calfactor=1
try:
    laser_percent
except NameError:
    print('laser_pecent is not defined and it is now 90!')
    laser_percent=84.3
percent_step=0.05 # default = 0.02
# laser_percent=84.3
sens=10 # in mV
sens2=20e-3 # in mV

###################### Set Directory
os.mkdir('D:/Jupyter_notebooks/Kyongmo/2308/BLS_%s'%fname)
ghost.write('override');time.sleep(3)
ghost.write('wdir D:/Jupyter_notebooks/Kyongmo/2308/BLS_%s'%fname)

###################### Save picture at starting position 
os.mkdir('D:/Jupyter_notebooks/Kyongmo/2308/Pictures/%s'%fname)
# takepic('D:/Jupyter_notebooks/Kyongmo/2308/Pictures/%s/begin.png'%fname)

###################### Register time
time_start=time.localtime()
print('start at : %d.%d.%d,%d:%d'%(time_start[0],time_start[1],time_start[2],time_start[3],time_start[4]))
time_start=time.time()

###################### Set swiches
sweepcount=1
dt=[]
error_occured=[]

#####################save codes
get_ipython().magic(u'history -n -f history.py')
num=read_linenum('history.py')
os.remove('history.py')
get_ipython().magic('%save -f temp.py -n {}'.format(str(num)))
save_header('temp.py',fname+'.dat', header)

####################### Loop
init_xpos=stagex.get_position()
init_ypos=stagey.get_position()

try:
    for f0,modfreq in enumerate(modfreq_list):
        print('Mod freq is changing...')
#         set_LFO_pulse(pulse_const/modfreq,1/modfreq)
#         sr830a.write('FREQ %.4f'%modfreq)
        time.sleep(1)

        for k1,laserpower in enumerate(laserpower_list):
            print('Laser power adjustment is starting...')
#             laser_percent_new=AdjustTorus(laserpower,laser_percent,percent_step)
#             time.sleep(1)
            
            for s2,ptcurr in enumerate(ptcurr_list):
#                 key2962A.write('CURR %3f'%ptcurr)
                time.sleep(0.01)
                
                for k2,power in enumerate(power_list):
                    print('MW Power is changing...')
                    change_power(power)
                    time.sleep(1)

                    for j,curr in enumerate(curr_list):
                        print('Kepco current is changing...')
                        change_curr_local(curr)
                        time.sleep(0.5) # stablization of field

                        for n,volt in enumerate(volt_list):
                            print('PSP voltage is changing...')
    #                         if n==0:
    #                             PSP_seq_volt(volt_list[-1]) # remove hystresis
    #                             PSP_seq_volt(volt_list[0]) # remove hystresis
    #                         PSP_seq_volt(volt)
#                             change_volt_PSP(volt)
#                             time.sleep(1) # stablization of field

                            for m1,hz in enumerate(hz_list):
#                                 move2field(**dict(desired_field=hz,step=1000,calfactor=1))
                                time.sleep(0.1)
    
                                for m3,posy in enumerate(posy_list):
                                    if m3==0:
                                        pass
                                    else:
                                        stagex.move_to(int(init_xpos));time.sleep(2)
                                        stagey.move_by(int(posy));time.sleep(1)
                                        print(f'y-stage is now at {stagey.get_position()}.')
                                        time.sleep(1)
                                        while stagey.is_moving()==1:
                                            time.sleep(0.5)
                                            print('waiting for y-stage to stop...')
                                        time.sleep(1)
                                            
                                    for m2,posx in enumerate(posx_list):
                                        if m2==0:
                                            pass
                                        else:
                                            stagex.move_to(int(init_xpos))
                                            print(f'x-stage is now at {stagex.get_position()}.')
                                            stagex.move_by(int(posx));time.sleep(1)
                                            time.sleep(1)
                                            while stagex.is_moving()==1:
                                                time.sleep(0.5)
                                                print('waiting for x-stage to stop...')
                                            time.sleep(1)
                                        
                                        time.sleep(2)    
                                        takepic('D:/Jupyter_notebooks/Kyongmo/2308/Pictures/%s/x=%i_y=%i.png'%(fname,m2,m3))
                                        
                                        for i,freq in enumerate(freq_list):
                                            time1=time.time()
                                            print('MW Freq is changing...')
                                            change_freq(freq)
                                            time.sleep(0.07)
        #                                     thorlabsPM16.write('READ?'); # read laser power
#                                             lpowernow=get_powerNOW()
                                            ghost.write('delete');time.sleep(0.1)
                                            print('starting scan number %i...'%sweepcount)
                                            ghost.write('start %i'%BLScount);#time.sleep(90/100*BLScount)
                                            error_occured=check_ghost_acquiring(sweepcount,BLScount);time.sleep(0.1)
                                            ghost.write('stop');time.sleep(0.1)
                                            ghost.write('save %i.dat'%sweepcount);time.sleep(0.1)

                                            print('%i.dat created'%sweepcount)
                                            print('scan number %i finished.'%sweepcount)
                                            Ncol=19
                                            data_raw=np.zeros(Ncol)
                                            data_raw[0] = float(power)
                                            data_raw[1] = stagex.get_position()
                                            data_raw[2] = stagey.get_position()
                                            data_raw[3] = hz                   
                                            data_raw[4] = float(curr)
                                            data_raw[5] = float(freq)
                                            data_raw[6] = float((kepco.query('meas:curr?')).split('\n')[0]) #actual current
                                            data_raw[7] = 0#float(getAvg(sens))#float(getAvg()) #lockin X voltage
                                            data_raw[8] = 0#lpowernow
                                            data_raw[9] = float(volt) # PSP voltage output
                                            data_raw[10] = 0#float(f41.query('FETCh:DC?'))*1e3*1 # Field at the center in mT : 0.9172=calfactor
                                            data_raw[11] = 0#float(laserpower) # laser power
                                            data_raw[12] = float(modfreq) # modulation 
                                            data_raw[13] = float(sens)
                                            data_raw[14] = float(time1) 
                                            data_raw[15] = float(error_occured)
                                            data_raw[16] = 0#float(getlock2x(sens2))
                                            data_raw[17] = 0#float(getlock2y(sens2))
                                            data_raw[18] = float(ptcurr)
                                            status=stopper('status.py')
                                            if status=='STOP':
                                                print('Measurement has been stopped')
                                                time.sleep(1)
                                                raise StopIteration

                                            data_arr=np.array([data_raw[0],data_raw[1],data_raw[2],\
                                                               data_raw[3],data_raw[4],data_raw[5],\
                                                               data_raw[6],data_raw[7],data_raw[8],\
                                                               data_raw[9],data_raw[10],data_raw[11],\
                                                               data_raw[12],data_raw[13],data_raw[14],\
                                                               data_raw[15],data_raw[16],data_raw[17],\
                                                              data_raw[18]])
                                            data = np.reshape(data_arr,(1,Ncol))
                                            if sweepcount==1:
                                                fp=open(fname+'.dat','a')
                                            else:
                                                pass
                                            np.savetxt(fp,data, delimiter='\t', newline=os.linesep);time.sleep(0.1)
                                            fp.flush() # flush all buffer data to a file
                                            TotN=len(curr_list)*len(power_list)*len(freq_list)*len(posx_list)*len(posy_list)*len(hz_list)\
                                            *len(volt_list)*len(laserpower_list)*len(modfreq_list)*len(ptcurr_list)
                                            if sweepcount==TotN:
                                                fp.close()  
                                            print('MW Power: {} dBm, Pos x: {}, Pos y: {}, Pos z: {}, Kepco : {} A, MW freq : {} Hz, Laser Power: {} mW, Field : {}, Mod Freq : {}'.format(\
                                                    data_raw[0],data_raw[1],data_raw[2],data_raw[3], data_raw[6], data_raw[5],data_raw[8],data_raw[10],data_raw[12]))
                                            time2=time.time()
                                            dt.append((time2-time1)/60)
                                            dtmean=np.mean(dt)
                                            tleft=dtmean*(TotN-sweepcount)
                                            print('Time left : %.4f minutes'%(tleft))
                                            print('Completed scan %d out of %d'%(sweepcount,TotN))
                                            sweepcount=sweepcount+1
                                            
                                            ########## safety checks ##########
                                            # check laser power
                                            #if (data_raw[8]>laserpower*1.005)|(data_raw[8]<laserpower*0.995):
                                             #   print('Laser power adjustment is starting...')
                                              #  laser_percent=AdjustTorus(laserpower,laser_percent,percent_step)
                                            
                                            # check scan stability
#                                             zerodata=get_zerodata('2308',fname)
#                                             latestdata=get_latestdata('2308',fname)
#                                             if (latestdata>zerodata*1.5)|(latestdata<zerodata*0.5):
#                                                 print('Scan is unstable!')
#                                                 laser_percent=AdjustTorus(0.07,laser_percent,1)
#                                                 time.sleep(1)
#                                                 raise StopIteration
#                                             else:
#                                                 print('Scan works fine!')
except StopIteration:
    fp.close()
    
###################### Save picture at ending position 
# takepic('D:/Jupyter_notebooks/Kyongmo/2308/Pictures/%s/end.png'%fname)
##return to original posiiton
# stagex.move_to(int(init_xpos))
# stagey.move_to(int(init_ypos))
print('Total time elapsed %.3f mins'%((time2-time_start)/60))
ghost.write('restore');time.sleep(1)

In [62]:
ghost.write('restore');time.sleep(1)

# Measure VNA

In [183]:
def setVNA(starfreq,stopfreq,pts,bw,power):
    pna5230a.write('SENS:FREQ:STAR %.9fGHz'%starfreq)
    pna5230a.write('SENS:FREQ:STOP %.9fGHz'%stopfreq)

    # number of points
    pna5230a.write('SENS:SWEEP:POINTS %i'%pts)
    # IF Bandwidth
    pna5230a.write('SENS:BAND %.9f'%bw)

    # Set power
    pna5230a.write('SOUR:POW %.9f'%power)
    # Set mode
#     pna5230a.write('CALC:PAR:SEL CH1_S11_1')
    pna5230a.write('CALC:PAR:SEL CH1_S11_1')
#     pna5230a.write('CALC:PAR:DEF MEAS2,S11')
#     pna5230a.write('DISP:WIND1:STATE ON')
    
def getVNAdata():
    pna5230a.write('SENS:SWE:GRO:COUNT 1')
    pna5230a.write('SENS:SWE:MODE GROUPS')
#     time.sleep(2)
#     pna5230a.write('*OPC?')
    time.sleep(7) # change this to 7 sec for 5001
    pna5230a.write('CALC:DATA? FDATA')
#     pna5230a.write('DISP:WIND1:TRAC:FEED MEAS1')
    
    data=pna5230a.read()
    data2=np.array([float(x) for x in data.split(',')])
    return data2

def setVNA_S21(starfreq,stopfreq,pts,bw,power):
    pna5230a.write('SENS:FREQ:STAR %.9fGHz'%starfreq)
    pna5230a.write('SENS:FREQ:STOP %.9fGHz'%stopfreq)

    # number of points
    pna5230a.write('SENS:SWEEP:POINTS %i'%pts)
    # IF Bandwidth
    pna5230a.write('SENS:BAND %.9f'%bw)

    # Set power
    pna5230a.write('SOUR:POW %.9f'%power)
    # Set mode
#     pna5230a.write('CALC:PAR:SEL CH1_S11_1')
#     pna5230a.write('CALC:PAR:SEL CH1_S11_1')
#     pna5230a.write('CALC:PAR:DEF MEAS2,S11')
#     pna5230a.write('DISP:WIND1:STATE ON')
    pna5230a.write("CALC:PAR:DEF 'MyMeas',S21")
    pna5230a.write("DISP:WIND:TRAC2:FEED 'MyMeas'")
    
def getVNAdata_S21():
    pna5230a.write('SENS:SWE:GRO:COUNT 1')
    pna5230a.write('SENS:SWE:MODE GROUPS')
#     time.sleep(2)
#     pna5230a.write('*OPC?')
    time.sleep(7) # change this to 7 sec for 5001
    pna5230a.write('CALC:DATA? FDATA')
#     pna5230a.write('DISP:WIND1:TRAC:FEED MEAS1')
    
    data=pna5230a.read()
    data2=np.array([float(x) for x in data.split(',')])
    return data2